# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt','wordnet'])

import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ritesh.Kumar6\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ritesh.Kumar6\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('tbl_disaster_response', con=engine)
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


### 2. Write a tokenization function to process your text data and prepare data

In [3]:
url_regex = r'https?://\S+|www\.\S+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() 
                    for tok in tokens]

    return clean_tokens

In [4]:
# prepare data
X = df.message
y = df.drop(columns=['id', 'message', 'original', 'genre'])
categories_list = y.columns
# categories_list

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(tokenizer=tokenize, max_features=1000)), # Convert text to TF-IDF features with tokenization and lemmatization
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=50, max_depth=10)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# split into test and train set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

# train pipeline
pipeline.fit(X_train, y_train)
# y_test

c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=1000,
                                 tokenizer=<function tokenize at 0x0000023FDEF254E0>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=10,
                                                                        n_estimators=50)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

for i in range(36):
    print("Category: ", y_test.columns[i])
    print(classification_report(y_test.iloc[:,i], y_pred[:,i], labels=[1,2,3], target_names=categories_list), '================================================================================')



Category:  related
                        precision    recall  f1-score   support

               related       0.76      1.00      0.86     15890
               request       0.00      0.00      0.00       143
                 offer       0.00      0.00      0.00         0

             micro avg       0.76      0.99      0.86     16033
             macro avg       0.25      0.33      0.29     16033
          weighted avg       0.76      0.99      0.86     16033
Category:  request
                        precision    recall  f1-score   support

               related       0.95      0.18      0.30      3595
               request       0.00      0.00      0.00         0
                 offer       0.00      0.00      0.00         0

             micro avg       0.95      0.18      0.30      3595
             macro avg       0.32      0.06      0.10      3595
          weighted avg       0.95      0.18      0.30      3595
Category:  offer
                        precision    recall  

c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-packages\sklearn\metrics\_classification.py:2642: UserWarning: labels size, 3, does not match size of target_names, 36
  warnings.warn(
c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-pack

                        precision    recall  f1-score   support

               related       0.00      0.00      0.00       231
               request       0.00      0.00      0.00         0
                 offer       0.00      0.00      0.00         0

             micro avg       0.00      0.00      0.00       231
             macro avg       0.00      0.00      0.00       231
          weighted avg       0.00      0.00      0.00       231
Category:  shops
                        precision    recall  f1-score   support

               related       0.00      0.00      0.00        95
               request       0.00      0.00      0.00         0
                 offer       0.00      0.00      0.00         0

             micro avg       0.00      0.00      0.00        95
             macro avg       0.00      0.00      0.00        95
          weighted avg       0.00      0.00      0.00        95
Category:  aid_centers
                        precision    recall  f1-score   supp

c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-packages\sklearn\metrics\_classification.py:2642: UserWarning: labels size, 3, does not match size of target_names, 36
  warnings.warn(
c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-pack

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__max_depth': [10, 20],
    'clf__estimator__min_samples_split': [2, 5],
    'clf__estimator__min_samples_leaf': [1, 2]
}

In [9]:
gridSearch = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1, verbose=2, cv=3)

In [10]:
gridSearch.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(max_features=1000,
                                                        tokenizer=<function tokenize at 0x0000023FDEF254E0>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=10,
                                                                                               n_estimators=50)))]),
             n_jobs=-1,
             param_grid={'clf__estimator__max_depth': [10, 20],
                         'clf__estimator__min_samples_leaf': [1, 2],
                         'clf__estimator__min_samples_split': [2, 5],
                         'clf__estimator__n_estimators': [50, 100]},
             verbose=2)

In [11]:
print(gridSearch.best_params_)

{'clf__estimator__max_depth': 20, 'clf__estimator__min_samples_leaf': 1, 'clf__estimator__min_samples_split': 5, 'clf__estimator__n_estimators': 50}


In [12]:
best_model = gridSearch.best_estimator_
print(best_model)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=1000,
                                 tokenizer=<function tokenize at 0x0000023FDEF254E0>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=20,
                                                                        min_samples_split=5,
                                                                        n_estimators=50)))])


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
y_pred = best_model.predict(X_test)

for i in range(36):
    print("Category: ", y_test.columns[i])
    print(classification_report(y_test.iloc[:,i], y_pred[:,i], labels=[1,2,3], target_names=categories_list), '================================================================================')




Category:  related
                        precision    recall  f1-score   support

               related       0.78      0.98      0.87     15890
               request       0.00      0.00      0.00       143
                 offer       0.00      0.00      0.00         0

             micro avg       0.78      0.98      0.87     16033
             macro avg       0.26      0.33      0.29     16033
          weighted avg       0.77      0.98      0.86     16033
Category:  request
                        precision    recall  f1-score   support

               related       0.91      0.36      0.52      3595
               request       0.00      0.00      0.00         0
                 offer       0.00      0.00      0.00         0

             micro avg       0.91      0.36      0.52      3595
             macro avg       0.30      0.12      0.17      3595
          weighted avg       0.91      0.36      0.52      3595
Category:  offer
                        precision    recall  

c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-packages\sklearn\metrics\_classification.py:2642: UserWarning: labels size, 3, does not match size of target_names, 36
  warnings.warn(
c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-pack

                        precision    recall  f1-score   support

               related       0.90      0.47      0.62      5900
               request       0.00      0.00      0.00         0
                 offer       0.00      0.00      0.00         0

             micro avg       0.90      0.47      0.62      5900
             macro avg       0.30      0.16      0.21      5900
          weighted avg       0.90      0.47      0.62      5900
Category:  floods
                        precision    recall  f1-score   support

               related       0.92      0.18      0.30      1735
               request       0.00      0.00      0.00         0
                 offer       0.00      0.00      0.00         0

             micro avg       0.92      0.18      0.30      1735
             macro avg       0.31      0.06      0.10      1735
          weighted avg       0.92      0.18      0.30      1735
Category:  storm
                        precision    recall  f1-score   support



c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Ritesh.Kumar6\AppData\Local\anaconda3\envs\dbconnect_11\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this beha

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [14]:
with open('Machine_classifier.pkl', 'wb') as file:
    pickle.dump(best_model, file)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.